<a href="https://colab.research.google.com/github/Fabiojeronymo/desafio-titanic-kaggle/blob/main/DesafioTitanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Leitura e tratamento inicial dos dados

In [46]:
# Importando a biblioteca pandas e lendos os arquivos csv
import pandas as pd

dados_teste = pd.read_csv("/content/test.csv")
dados_treino = pd.read_csv("/content/train.csv")

In [47]:
# Salvando a coluna PassengerId que o Kaggle exige
teste_passenger_ids = dados_teste['PassengerId']

# Removendo colunas que não vou utilizar
dados_teste = dados_teste.drop(['Name', 'Ticket', 'Cabin', 'PassengerId'], axis=1)
dados_treino = dados_treino.drop(['Name', 'Ticket', 'Cabin', 'PassengerId'], axis=1)

In [48]:
dados_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [49]:
# Tratando valores nulos do df de treino
# Tratando a coluna idade, usando a mediana de cada classe, se todos dados da classe for nulo ele usa mediana geral
dados_treino['Age'] = dados_treino.groupby('Pclass')['Age'].transform(lambda x: x.fillna(x.median()))
if dados_treino['Age'].isnull().any():
  dados_treino['Age'].fillna(dados_treino['Age'].median(), inplace = True)

# Dropando as 2 linhas do local de embarque
dados_treino.dropna(axis=0, how='any', inplace=True)

In [50]:
dados_treino.info()

<class 'pandas.core.frame.DataFrame'>
Index: 889 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  889 non-null    int64  
 1   Pclass    889 non-null    int64  
 2   Sex       889 non-null    object 
 3   Age       889 non-null    float64
 4   SibSp     889 non-null    int64  
 5   Parch     889 non-null    int64  
 6   Fare      889 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 62.5+ KB


In [51]:
dados_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    object 
 2   Age       332 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      417 non-null    float64
 6   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 23.0+ KB


In [53]:
# Tratando valores nulos do df de teste
dados_teste['Age'] = dados_teste.groupby('Pclass')['Age'].transform(lambda x: x.fillna(x.median()))
if dados_teste['Age'].isnull().any():
  dados_teste['Age'].fillna(dados_teste['Age'].median(), inplace = True)

# Tratando o valor nulo do valor da passagem usando a media geral
dados_teste['Fare'] = dados_teste['Fare'].fillna(dados_teste['Age'].median())

In [55]:
dados_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    object 
 2   Age       418 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      418 non-null    float64
 6   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 23.0+ KB


## Explorando variaveis qualitativas

In [77]:
# Importando biblioteca para plotar os graficos
import plotly.express as px

In [94]:
# Alterando a coluna Sovreviventes de 0 e 1 para Sobreviveu e Nao sobreviveu para melhor visualizacao
df_plot = dados_treino.copy()
df_plot['survived_label'] = df_plot['Survived'].map({0: 'Não sobreviveu', 1: 'Sobreviveu'})

# Alterando a coluna Sexo para portugues
df_plot['sex_label'] = df_plot['Sex'].map({'male': 'Homens', 'female': 'Mulheres'})

In [112]:
fig = px.histogram(df_plot, x = 'sex_label',
                  text_auto = True,
                  color = 'survived_label',
                  barmode = 'group',
                  title='Contagem de sobreviventes por Sexo')
fig.update_layout(
    xaxis_title = 'Sexo dos passageiros',
    yaxis_title = 'Número de Passageiros',
    legend_title_text='Status'
)

In [101]:
# Alterando a coluna de embarque
df_plot['Embarked_label'] = df_plot['Embarked'].map({'C': 'Cherbourg', 'Q': 'Queenstown', 'S': 'Southampton'})

In [113]:
fig = px.histogram(df_plot, x = 'Embarked_label',
                  text_auto = True,
                  color = 'survived_label',
                  barmode = 'group',
                  title='Contagem de sobreviventes por Local de embarque')
fig.update_layout(
    xaxis_title = 'Porto de Embarque',
    yaxis_title = 'Número de Passageiros',
    legend_title_text='Status'
)

In [105]:
# Alterando a coluna de ticket class
df_plot['Pclass_label'] = df_plot['Pclass'].map({1: 'Primeira', 2: 'Segunda', 3: 'Terceira'})

In [120]:
fig = px.histogram(df_plot, x = 'Pclass_label',
                  color = 'survived_label',
                  barnorm = 'percent',
                  text_auto = '.2f',
                  title='Taxa de sobreviventes por tipo de bilhete(classe)')
fig.update_layout(
    xaxis_title = 'Classe',
    yaxis_title = 'Porcentagem de Passageiros',
    legend_title_text='Status'
)

## Váriaveis quantitativas

In [115]:
fig = px.histogram(df_plot, x = 'Parch',
                  text_auto = True,
                  color = 'survived_label',
                  barmode = 'group',
                  title='Contagem de Sobrevivência por Nº de Pais/Filhos a Bordo',
                  labels={
                          'Parch': 'Número de Pais/Filhos a Bordo',
                          'count': 'Número de Passageiros',
                          'survived_label': 'Status'
                        })
fig.update_layout(
    xaxis_title = 'Número de Pais/Filhos a Bordo',
    yaxis_title = 'Total de Passageiros',
    legend_title_text='Status'
)

In [116]:
fig = px.histogram(df_plot, x = 'SibSp',
                  text_auto = True,
                  color = 'survived_label',
                  barmode = 'group',
                  title='Contagem de Sobrevivência por Nº de irmãos/cônjuges a Bordo',
                  labels={
                          'Parch': 'Número de irmãos/cônjuges a Bordo',
                          'count': 'Número de Passageiros',
                          'survived_label': 'Status'
                        })
fig.update_layout(
    xaxis_title = 'Número de irmãos/cônjuges a Bordo',
    yaxis_title = 'Total de Passageiros',
    legend_title_text='Status'
)

In [126]:
fig = px.box(df_plot,
             x = 'survived_label',
             y = 'Age',
             color = 'survived_label',
             title = 'Distribuição de Idade entre Sobreviventes e Não Sobreviventes',
             points = 'all')
fig.update_layout(xaxis_title = 'Status de sobrevivencia',
                  yaxis_title = 'Idade',
                  legend_title_text = 'Status')



## Construindo pré-processamento (One-hot Encoding)

In [56]:
# Separação de features x e alvo y
x_treino = dados_treino.drop('Survived', axis = 1)
y_treino = dados_treino['Survived']

In [57]:
# Concatenando os df para garantir consistencia no pré-processing
df_combinado = pd.concat([x_treino, dados_teste], axis=0, ignore_index=True)

In [58]:
# Importando bibliotecas necessarias
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [59]:
# Preparação do OneHotEncoder
colunas = x_treino.columns

one_hot = make_column_transformer((
    OneHotEncoder(drop = 'if_binary'),
    ['Pclass', 'Sex', 'Embarked']
),
  remainder = 'passthrough',
  sparse_threshold = 0)

In [60]:
# Transformação e reconstrução do DataFrame
df_combinado_array = one_hot.fit_transform(df_combinado)
novas_colunas = one_hot.get_feature_names_out()
df_combinado_processado = pd.DataFrame(df_combinado_array, columns=novas_colunas)

In [61]:
df_combinado_processado

,onehotencoder__Pclass_1,onehotencoder__Pclass_2,onehotencoder__Pclass_3,onehotencoder__Sex_male,onehotencoder__Embarked_C,onehotencoder__Embarked_Q,onehotencoder__Embarked_S,remainder__Age,remainder__SibSp,remainder__Parch,remainder__Fare
0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,22.0,1.0,0.0,7.2500
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,38.0,1.0,0.0,71.2833
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,26.0,0.0,0.0,7.9250
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,35.0,1.0,0.0,53.1000
4,0.0,0.0,1.0,1.0,0.0,0.0,1.0,35.0,0.0,0.0,8.0500
...,...,...,...,...,...,...,...,...,...,...,...
1302,0.0,0.0,1.0,1.0,0.0,0.0,1.0,24.0,0.0,0.0,8.0500
1303,1.0,0.0,0.0,0.0,1.0,0.0,0.0,39.0,0.0,0.0,108.9000
1304,0.0,0.0,1.0,1.0,0.0,0.0,1.0,38.5,0.0,0.0,7.2500
1305,0.0,0.0,1.0,1.0,0.0,0.0,1.0,24.0,0.0,0.0,8.0500


In [62]:
# Separando os dfs
# Pegando o numero de linhas original do x_treino para saber onde cortar
n_treino = len(x_treino)

# Separando os dados
x_treino = df_combinado_processado[:n_treino]
x_teste = df_combinado_processado[n_treino:]

# Criando as variaveis que vou usar no modelo final
x_teste_final = x_teste.copy()
x_treino_final = x_treino.copy()
y_treino_final = y_treino.copy()

In [63]:
x_teste_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 889 to 1306
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   onehotencoder__Pclass_1    418 non-null    float64
 1   onehotencoder__Pclass_2    418 non-null    float64
 2   onehotencoder__Pclass_3    418 non-null    float64
 3   onehotencoder__Sex_male    418 non-null    float64
 4   onehotencoder__Embarked_C  418 non-null    float64
 5   onehotencoder__Embarked_Q  418 non-null    float64
 6   onehotencoder__Embarked_S  418 non-null    float64
 7   remainder__Age             418 non-null    float64
 8   remainder__SibSp           418 non-null    float64
 9   remainder__Parch           418 non-null    float64
 10  remainder__Fare            418 non-null    float64
dtypes: float64(11)
memory usage: 36.1 KB


In [64]:
# Visualizando os dados novamente
print(f"Shape do X de treino final: {x_treino.shape}")
print(f"Shape do X de teste final: {x_teste.shape}")

Shape do X de treino final: (889, 11)
Shape do X de teste final: (418, 11)


## Treinamento e avaliação dos modelos

In [65]:
# Separando os dados em validação e teste
from sklearn.model_selection import train_test_split

x_treino, x_valid, y_treino, y_valid = train_test_split(x_treino,
                                                        y_treino,
                                                        stratify = y_treino,
                                                        test_size = 0.15)

In [66]:
# Modelo base
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier()
dummy.fit(x_treino, y_treino)

print(f'Acurácia de treino: {dummy.score(x_treino, y_treino)}')
print(f'Acurácia de validação: {dummy.score(x_valid, y_valid)}')

Acurácia de treino: 0.6172185430463576
Acurácia de validação: 0.6194029850746269


In [67]:
# Arvore de decisão
from sklearn.tree import DecisionTreeClassifier

arvore = DecisionTreeClassifier(max_depth=7)
arvore.fit(x_treino, y_treino)

print(f'Acurácia de treino: {arvore.score(x_treino, y_treino)}')
print(f'Acurácia de validação: {arvore.score(x_valid, y_valid)}')

Acurácia de treino: 0.8847682119205298
Acurácia de validação: 0.8582089552238806


In [68]:
# Normalização dos dados para knn
# Biblioteca nessaria
from sklearn.preprocessing import MinMaxScaler
normalizacao = MinMaxScaler()
x_treino_normalizado = normalizacao.fit_transform(x_treino)

In [69]:
# Knn
from sklearn.neighbors import KNeighborsClassifier

# Baseline
knn = KNeighborsClassifier()

# Metodo do modelo que ele aprende com os dados
knn.fit(x_treino_normalizado, y_treino)
x_valid_normalizado = normalizacao.transform(x_valid)

# Executa a validação
print(f'Acurácia de treino: {knn.score(x_treino_normalizado, y_treino)}')
print(f'Acurácia de validação: {knn.score(x_valid_normalizado, y_valid)}')

Acurácia de treino: 0.8503311258278146
Acurácia de validação: 0.8208955223880597


## Modelo Final

In [70]:
normalizador_final = MinMaxScaler()
x_treino_final_normalizado = normalizador_final.fit_transform(x_treino_final)

knn_final = KNeighborsClassifier()
knn_final.fit(x_treino_final_normalizado, y_treino_final)

KNeighborsClassifier()

In [71]:
x_teste_final_normalizado = normalizador_final.transform(x_teste_final)

In [72]:
# Previsão final
previsoes = knn_final.predict(x_teste_final_normalizado)
print("Previsões para o conjunto de teste foram realizadas!")
print(previsoes[:10])

Previsões para o conjunto de teste foram realizadas!
[0 0 1 1 0 0 0 0 1 0]


## Criação do arquivo de submissão para o Kaggle

In [73]:
# Criação do arquivo para o Kaggle

# Id do passageiro salvo no inicio e as previsoes
submissao = pd.DataFrame({
    'PassengerId': teste_passenger_ids,
    'Survived': previsoes
})

# Salvar o arquivo como csv sem o indice do pandas
submissao.to_csv('desafio_titanic_knn.csv', index = False)

In [74]:
sub = pd.read_csv('/content/desafio_titanic_knn.csv')

In [75]:
sub

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
